In [1]:
import numpy as np
import pandas as pd
from PyPDF2 import PdfFileReader

# Load the PDF File

In [2]:
# open the PDF file
pdfFile = open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb')


# Read the File

In [3]:
# create PDFFileReader object to read the file
pdfReader = PdfFileReader(pdfFile)

# Check For Encryption

In [4]:
if pdfReader.isEncrypted == True:
    print("PDF File is encrypted. Password is needed to continue.")
else:
    print("PDF File is not encrypted. Password is not needed.")

PDF File is encrypted. Password is needed to continue.


# Print Document Info

In [5]:
#print("PDF File name: " + str(pdfReader.getDocumentInfo().title))
#print("PDF File created by: " + str(pdfReader.getDocumentInfo().creator))
#print("- - - - - - - - - - - - - - - - - - - -")

# Try Using PDFminer3

In [6]:
#imports
import pandas as pd

from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

In [9]:
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 1,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        7815-OTH           02/22/1985 1                                                                  

        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            
        MENDEZ, OLGA LIDIA                                                                                         
        -------------------------------------------------------------------------------------------------------------
        90-11-10256-CV     11/08/1990 1                                                                  

        ATTORNEY GENERAL            

In [10]:
#Now try finding places to break it down a bit
print(repr(text))

'                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GEN

In [12]:
#First, strip leading and trailing whitespaces
text = text.strip()
print(repr(text))

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL            

In [22]:
#Now remove header info
text = text[392:]
print(repr(text))

'7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        -------------------------------------------------------------------------------------------------------------\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE OTHER T                     

In [23]:
#Now split each case by the dashed lines
cases = text.split('-------------------------------------------------------------------------------------------------------------')
cases

['7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        ',
 '\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        ',
 '\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE OTHER T                                           \n\n        DIAZ CARPENTER, ENRIQUETA     JAY T. KIMBROUGH          BOWLES JR., RUDY              RICHARD TINSMAN      \n        ',
 '\n        91-06-10598-CV     10/22/1993 MOTION ENFO

In [27]:
test_case = cases[0]
print(repr(test_case))

'7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        '
